In [1]:
import pandas as pd

df = pd.read_csv('movie_dataset_classification.csv',index_col=0)
df.dropna(subset=['Plot'], inplace=True)
df.drop_duplicates(subset=['Title','Plot'], inplace=True)
#removing low frequency genres
counts = df.Genre.value_counts()
counts = list(counts[counts < 150].index)
df = df[~df['Genre'].isin(counts)]


In [2]:
# pip install -U numpy==1.18.5
# con numpy successivo dà problemi

Note: you may need to restart the kernel to use updated packages.


In [3]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

max_features = 5000
maxlen = 100
batch_size = 1000
embedding_dims = 10
hidden_dims = 20

In [4]:
from sklearn.model_selection import train_test_split

X = df['Plot'].values
y = df['Genre'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, shuffle=True)

In [8]:
import numpy as np 

sample_idx = 0
y_train_bin = np.asarray(y_train)==y_train[sample_idx]
y_test_bin = np.asarray(y_test)==y_train[sample_idx]
y_train_bin,y_test_bin

(array([ True, False, False, ..., False, False, False]),
 array([False, False, False, ..., False, False, False]))

In [11]:
df.dtypes

Release Year    float64
Title            object
Director         object
Cast             object
Plot             object
Genre            object
dtype: object

In [12]:
from sklearn.preprocessing import LabelEncoder

target=df['Genre'].values.tolist()
label_encoder = LabelEncoder()
Y = np.array(label_encoder.fit_transform(target))

In [14]:
df.dtypes

Release Year    float64
Title            object
Director         object
Cast             object
Plot             object
Genre            object
dtype: object

In [5]:
model = Sequential()
model.add(Embedding(max_features, embedding_dims))
model.add(LSTM(hidden_dims, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 10)          50000     
_________________________________________________________________
lstm (LSTM)                  (None, 20)                2480      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 52,501
Trainable params: 52,501
Non-trainable params: 0
_________________________________________________________________


### Test embedding

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=embedding_dims
)


In [38]:
model = Sequential()

In [39]:
from tensorflow.keras import Input 
from tensorflow import string 
model.add(Input(shape=(1,), dtype=string))

In [40]:
model.add(vectorize_layer)

In [33]:
input_data = [["foo qux bar"], ["qux baz"]]

In [34]:
model.predict(input_data)

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [17]:
# Embedding(X_train, embedding_dims)

TypeError: '<=' not supported between instances of 'str' and 'int'

In [15]:
epochs = 10
print('Train...')
model.fit(X_train, Y,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          validation_split=0.1)

Train...
Epoch 1/10


UnimplementedError:  Cast string to float is not supported
	 [[node sequential/Cast (defined at \AppData\Local\Temp/ipykernel_9728/2052679473.py:3) ]] [Op:__inference_train_function_7740]

Function call stack:
train_function


In [ ]:
predictions = model.predict(x_test)